## [Introduction to Cython for Solving Differential Equations](http://hplgit.github.io/teamods/cyode/cyode-sphinx/main_cyode.html) 
-  part two

### Using arrays

In [2]:
import time
import timeit

In [29]:
N = 300000

In [36]:
import numpy as np
import pylab as pl

def solver(f, I, t, method):
    t = np.asarray(t)
    N = len(t)-1
    u = np.zeros(N+1)
    u[0] = I

    for n in range(N):
        u[n+1] = method(u, n, t, f)
    return u, t

def RK2(u, n, t, f):
    dt = t[n+1] - t[n]
    K1 = dt*f(u[n], t[n])
    K2 = dt*f(u[n] + 0.5*K1, t[n] + 0.5*dt)
    unew = u[n] + K2
    return unew

def problem1(u, t):
    return - u + 1

def problem2(u, t):
    return - u + exp(-2*t)

In [37]:
def run(N, problem, plot=False):
        I = 0
        t = np.linspace(0., 5., N)
        u, t = solver(problem, I, t, RK2)
        
        if plot:
            print(len(u), len(t))
            pl.plot(t, u)
            pl.show()

In [41]:
# run pure python 
%timeit run(N, problem1)
%timeit run(N, problem2)

1.31 s ± 38.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.46 s ± 3.09 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [42]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [51]:
%%cython
import numpy as np
import pylab as pl

def solver(f, I, t, method):
    t = np.asarray(t)
    N = len(t)-1
    u = np.zeros(N+1)
    u[0] = I

    for n in range(N):
        u[n+1] = method(u, n, t, f)
    return u, t

def RK2(u, n, t, f):
    dt = t[n+1] - t[n]
    K1 = dt*f(u[n], t[n])
    K2 = dt*f(u[n] + 0.5*K1, t[n] + 0.5*dt)
    unew = u[n] + K2
    return unew

def problem1(u, t):
    return - u + 1

from math import exp
def problem2(u, t):
    return - u + exp(-2*t)


In [52]:
# only compiled with cython 
%timeit run(N, problem1)
%timeit run(N, problem2)

1.09 s ± 109 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.24 s ± 18.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [53]:
%%cython
"""
Variables are declared with types.
Functions as arguments are represented by classes and instances.
"""
# Note: need both numpy imports!
import numpy as np
cimport numpy as np

cdef class Problem:
    cpdef double rhs(self, double u, double t):
        return 0

cdef class Problem1(Problem):
    cpdef double rhs(self, double u, double t):
        return -u + 1  # u = 1-exp(-t)

from math import exp

cdef class Problem2(Problem):
    cpdef double rhs(self, double u, double t):
        return - u + exp(-2*t)

cdef class ODEMethod:
    cpdef double advance(self, np.ndarray u, int n,
                         np.ndarray t, Problem p):
        return 0

cdef class Method_RK2(ODEMethod):
    cpdef double advance(self, np.ndarray u, int n,
                         np.ndarray t, Problem p):
        cdef double K1, K2, unew, dt
        dt = t[n+1] - t[n]
        K1 = dt*p.rhs(u[n], t[n])
        K2 = dt*p.rhs(u[n] + 0.5*K1, t[n] + 0.5*dt)
        unew = u[n] + K2
        return unew

# Create names compatible with ode0.py
RK2 = Method_RK2()
problem1 = Problem1()
problem2 = Problem2()

cpdef solver(Problem f, double I, np.ndarray t, ODEMethod method):
    cdef int N = len(t)-1
    cdef np.ndarray u = np.zeros(N+1, dtype=np.float)
    u[0] = I

    cdef int n
    for n in range(N):
        u[n+1] = method.advance(u, n, t, f)
    return u, t


In [54]:
%timeit run(N, problem1)
%timeit run(N, problem2)

443 ms ± 117 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
601 ms ± 13.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [55]:
# declaring numpy array with type and dimension
# np.ndarray[np.float_t, ndim=1]

In [57]:
%%cython
import numpy as np   # note: need both imports!
cimport numpy as np

cdef class Problem:
    cpdef double rhs(self, double u, double t):
        return 0

cdef class Problem1(Problem):
    cpdef double rhs(self, double u, double t):
        return -u +1  # u = 1-exp(-t)

# cdef extern from "math.h":
#     double exp(double)
from math import exp

cdef class Problem2(Problem):
    cpdef double rhs(self, double u, double t):
        return - u + exp(-2*t)

# NOTE: need def, not cpdef, for functions with array arguments
# and [] buffer notation.
# Common error message: "Expected ']'"

cdef class ODEMethod:
    def advance(self, 
                np.ndarray[np.float_t, ndim=1] u,
                int n, 
                np.ndarray[np.float_t, ndim=1] t, 
                Problem p):
        return 0

cdef class Method_RK2(ODEMethod):
    def advance(self, 
                np.ndarray[np.float_t, ndim=1] u, 
                int n,
                np.ndarray[np.float_t, ndim=1] t, 
                Problem p):
        """2nd-orderRunge-Kutta method."""
        cdef double K1, K2, unew, dt
        dt = t[n+1] - t[n]
        K1 = dt*p.rhs(u[n], t[n])
        K2 = dt*p.rhs(u[n] + 0.5*K1, t[n] + 0.5*dt)
        unew = u[n] + K2
        return unew
    
# Create names compatible with ode0.py
RK2 = Method_RK2()
problem1 = Problem1()
problem2 = Problem2()


def solver(Problem f, double I, 
           np.ndarray[np.float_t, ndim=1] t, 
           ODEMethod method):
    cdef int N = len(t)-1
    #cdef np.ndarray[np.float_t, ndim=1] u = np.zeros(N+1, dtype=np.float_t)
    #Cython does not like type specification via dtype when the buffer
    #declares the type
    cdef np.ndarray[np.float_t, ndim=1] u = np.zeros(N+1)
    u[0] = I   
             
    cdef int n
    for n in range(N):
        u[n+1] = method.advance(u, n, t, f)
    return u, t

In [58]:
%timeit run(N, problem1)
%timeit run(N, problem2)

254 ms ± 30.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
335 ms ± 1.38 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Our RK2.advance method takes array arguments and performs operations on two single array elements u[n] and t[n]. We could easily avoid this and instead transfer u[n] and t[n] as double arguments

In [60]:
%%cython
import numpy as np   # note: need both imports!
cimport numpy as np

cdef class Problem:
    cpdef double rhs(self, double u, double t):
        return 0

cdef class Problem1(Problem):
    cpdef double rhs(self, double u, double t):
        return -u +1  # u = 1-exp(-t)

from math import exp
# cdef extern from "math.h":
#     double exp(double)

#or
#from libc.math cimport exp  # may need explicit -lm linking
#see http://docs.cython.org/src/tutorial/external.html

cdef class Problem2(Problem):
    cpdef double rhs(self, double u, double t):
        return - u + exp(-2*t)

# NOTE: need def, not cpdef, for functions with array arguments
# and [] buffer notation.
# This means that def functions with arrays are not called very
# efficiently, and the RK2.advance function, which basically
# works with a single array element should be implemented alternatively
# via doubles only.

cdef class ODEMethod:
    cpdef advance(self, double u_1, int n, double t_1,
                  double dt, Problem p):
        return 0

cdef class Method_RK2(ODEMethod):
    cpdef advance(self, double u_1, int n, double t_1,
                  double dt, Problem p):
        cdef double K1, K2, unew
        K1 = dt*p.rhs(u_1, t_1)
        K2 = dt*p.rhs(u_1 + 0.5*K1, t_1 + 0.5*dt)
        unew = u_1 + K2
        return unew
    
# Create names compatible with ode0.py
RK2 = Method_RK2()
problem1 = Problem1()
problem2 = Problem2()


def solver(Problem f, double I, 
           np.ndarray[np.float_t, ndim=1] t, 
           ODEMethod method):
    cdef int N = len(t)-1
    #cdef np.ndarray[np.float_t, ndim=1] u = np.zeros(N+1, dtype=np.float_t)
    #Cython does not like type specification via dtype when the buffer
    #declares the type
    cdef np.ndarray[np.float_t, ndim=1] u = np.zeros(N+1)
    u[0] = I   
             
    cdef int n
    for n in range(N):
        u[n+1] = method.advance(u[n], n, t[n], t[n+1]-t[n], f)
    return u, t

In [61]:
%timeit run(N, problem1)
%timeit run(N, problem2)

9.51 ms ± 2.82 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
67.8 ms ± 767 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [62]:
# turn off bounds checking for this func.

In [63]:
%%cython
import numpy as np 
cimport numpy as np
cimport cython

cdef class Problem:
    cpdef double rhs(self, double u, double t):
        return 0

cdef class Problem1(Problem):
    cpdef double rhs(self, double u, double t):
        return -u +1  # u = 1-exp(-t)

from math import exp
# cdef extern from "math.h":
#     double exp(double)

cdef class Problem2(Problem):
    cpdef double rhs(self, double u, double t):
        return - u + exp(-2*t)

ctypedef np.float64_t DT

cdef class ODEMethod:
    cpdef advance(self, double u_1, int n, double t_1,
                  double dt, Problem p):
        return 0

cdef class Method_RK2(ODEMethod):
    cpdef advance(self, double u_1, int n, double t_1,
                  double dt, Problem p):
        cdef double K1, K2, unew
        K1 = dt*p.rhs(u_1, t_1)
        K2 = dt*p.rhs(u_1 + 0.5*K1, t_1 + 0.5*dt)
        unew = u_1 + K2
        return unew
    
# Create names compatible with ode1.py
RK2 = Method_RK2()
problem1 = Problem1()
problem2 = Problem2()

@cython.boundscheck(False) # turn off bounds checking for this func.
def solver(Problem f, 
           double I, 
           np.ndarray[DT, ndim=1, negative_indices=False, 
                      mode='c'] t, 
           ODEMethod method):
    cdef int N = len(t)-1
    #cdef np.ndarray[DT, ndim=1, negative_indices=False, mode='c'] u = np.zeros(N+1, dtype=np.float_t)
    #Cython does not like type specification via dtype when the buffer
    #declares the type
    cdef np.ndarray[DT, ndim=1, negative_indices=False, 
                    mode='c'] u = np.zeros(N+1)
    u[0] = I   
             
    cdef int n
    for n in range(N):
        u[n+1] = method.advance(u[n], n, t[n], t[n+1]-t[n], f)
    return u, t

In [64]:
%timeit run(N, problem1)
%timeit run(N, problem2)

9.02 ms ± 2.77 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
47.5 ms ± 16.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Changing the data type `double` to `np.float_t` all over the Cython code has negligible effect

In [65]:
%%cython
import numpy as np 
cimport numpy as np
cimport cython

cdef class Problem:
    cpdef np.float_t rhs(self, np.float_t u, np.float_t t):
        return 0

cdef class Problem1(Problem):
    cpdef np.float_t rhs(self, np.float_t u, np.float_t t):
        return -u +1  # u = 1-exp(-t)

from math import exp
# cdef extern from "math.h":
#     np.float_t exp(np.float_t)

cdef class Problem2(Problem):
    cpdef np.float_t rhs(self, np.float_t u, np.float_t t):
        return - u + exp(-2*t)

ctypedef np.float64_t DT

cdef class ODEMethod:
    cpdef advance(self, np.float_t u_1, int n, np.float_t t_1,
                  np.float_t dt, Problem p):
        return 0

cdef class Method_RK2(ODEMethod):
    cpdef advance(self, np.float_t u_1, int n, np.float_t t_1,
                  np.float_t dt, Problem p):
        cdef np.float_t K1, K2, unew
        K1 = dt*p.rhs(u_1, t_1)
        K2 = dt*p.rhs(u_1 + 0.5*K1, t_1 + 0.5*dt)
        unew = u_1 + K2
        return unew
    
# Create names compatible with ode1.py
RK2 = Method_RK2()
problem1 = Problem1()
problem2 = Problem2()

@cython.boundscheck(False) # turn off bounds checking for this func.
def solver(Problem f, 
           np.float_t I, 
           np.ndarray[DT, ndim=1, negative_indices=False, mode='c'] t, 
           ODEMethod method):
    cdef int N = len(t)-1
    #cdef np.ndarray[DT, ndim=1, negative_indices=False, mode='c'] u = np.zeros(N+1, dtype=np.float_t)
    #Cython does not like type specification via dtype when the buffer
    #declares the type
    cdef np.ndarray[DT, ndim=1, negative_indices=False, 
                    mode='c'] u = np.zeros(N+1)
    u[0] = I   
             
    cdef int n
    for n in range(N):
        u[n+1] = method.advance(u[n], n, t[n], t[n+1]-t[n], f)
    return u, t

In [66]:
%timeit run(N, problem1)
%timeit run(N, problem2)

10.2 ms ± 2.45 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
48.2 ms ± 16.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [67]:
# using `exp` from `math.h`

In [68]:
%%cython
import numpy as np 
cimport numpy as np
cimport cython

cdef class Problem:
    cpdef double rhs(self, double u, double t):
        return 0

cdef class Problem1(Problem):
    cpdef double rhs(self, double u, double t):
        return -u +1  # u = 1-exp(-t)

# from math import exp
cdef extern from "math.h":
    double exp(double)

cdef class Problem2(Problem):
    cpdef double rhs(self, double u, double t):
        return - u + exp(-2*t)

ctypedef np.float64_t DT

cdef class ODEMethod:
    cpdef advance(self, double u_1, int n, double t_1,
                  double dt, Problem p):
        return 0

cdef class Method_RK2(ODEMethod):
    cpdef advance(self, double u_1, int n, double t_1,
                  double dt, Problem p):
        cdef double K1, K2, unew
        K1 = dt*p.rhs(u_1, t_1)
        K2 = dt*p.rhs(u_1 + 0.5*K1, t_1 + 0.5*dt)
        unew = u_1 + K2
        return unew
    
# Create names compatible with ode1.py
RK2 = Method_RK2()
problem1 = Problem1()
problem2 = Problem2()

@cython.boundscheck(False) # turn off bounds checking for this func.
def solver(Problem f, 
           double I, 
           np.ndarray[DT, ndim=1, negative_indices=False, 
                      mode='c'] t, 
           ODEMethod method):
    cdef int N = len(t)-1
    #cdef np.ndarray[DT, ndim=1, negative_indices=False, mode='c'] u = np.zeros(N+1, dtype=np.float_t)
    #Cython does not like type specification via dtype when the buffer
    #declares the type
    cdef np.ndarray[DT, ndim=1, negative_indices=False, 
                    mode='c'] u = np.zeros(N+1)
    u[0] = I   
             
    cdef int n
    for n in range(N):
        u[n+1] = method.advance(u[n], n, t[n], t[n+1]-t[n], f)
    return u, t

In [69]:
%timeit run(N, problem2)

16.7 ms ± 6.26 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
